**using imports and packages/

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Load the diabetes dataset (Note: this is actually a regression dataset)
data = load_diabetes()
X = data.data
y_continuous = data.target

# Convert regression target into binary classification:
# Let's say target > 140 is considered "diabetic" (1), else "non-diabetic" (0)
y = (y_continuous > 140).astype(int)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Output results
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Example prediction
sample = X_test_scaled[0].reshape(1, -1)
sample_pred = model.predict(sample)
print("\nSample prediction (0 = non-diabetic, 1 = diabetic):", sample_pred[0])


Confusion Matrix:
 [[36 13]
 [11 29]]

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.73      0.75        49
           1       0.69      0.72      0.71        40

    accuracy                           0.73        89
   macro avg       0.73      0.73      0.73        89
weighted avg       0.73      0.73      0.73        89

Accuracy Score: 0.7303370786516854

Sample prediction (0 = non-diabetic, 1 = diabetic): 0


#without imports an packages#

In [5]:
# Sigmoid function
def sigmoid(z):
    if z < -100:
        return 0  # Prevent underflow
    if z > 100:
        return 1  # Prevent overflow
    return 1 / (1 + (2.71828 ** -z))

# Cost function (Log Loss)
def compute_cost(X, y, weights):
    m = len(y)
    cost = 0
    for i in range(m):
        z = sum(X[i][j] * weights[j] for j in range(len(weights)))
        pred = sigmoid(z)
        # Avoid log(0) by clipping prediction values
        pred = max(min(pred, 1 - 1e-15), 1e-15)
        cost += -(y[i] * math.log(pred) + (1 - y[i]) * math.log(1 - pred))
    return cost / m

# Gradient Descent
def gradient_descent(X, y, weights, learning_rate, iterations):
    m = len(y)
    for it in range(iterations):
        gradients = [0] * len(weights)
        for i in range(m):
            z = sum(X[i][j] * weights[j] for j in range(len(weights)))
            pred = sigmoid(z)
            error = pred - y[i]
            for j in range(len(weights)):
                gradients[j] += error * X[i][j]
        for j in range(len(weights)):
            weights[j] -= learning_rate * gradients[j] / m
        if it % 100 == 0:
            print("Iteration", it, ": Cost =", compute_cost(X, y, weights))
    return weights

# Prediction function
def predict(X, weights):
    predictions = []
    for i in range(len(X)):
        z = sum(X[i][j] * weights[j] for j in range(len(weights)))
        prob = sigmoid(z)
        predictions.append(1 if prob >= 0.5 else 0)
    return predictions

# Evaluation metrics
def evaluate(y_true, y_pred):
    tp = sum(1 for i in range(len(y_true)) if y_true[i] == 1 and y_pred[i] == 1)
    tn = sum(1 for i in range(len(y_true)) if y_true[i] == 0 and y_pred[i] == 0)
    fp = sum(1 for i in range(len(y_true)) if y_true[i] == 0 and y_pred[i] == 1)
    fn = sum(1 for i in range(len(y_true)) if y_true[i] == 1 and y_pred[i] == 0)

    accuracy = (tp + tn) / len(y_true)
    precision = tp / (tp + fp) if (tp + fp) else 0
    recall = tp / (tp + fn) if (tp + fn) else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    print("Accuracy:", round(accuracy, 4))
    print("Precision:", round(precision, 4))
    print("Recall:", round(recall, 4))
    print("F1 Score:", round(f1, 4))

# Load dataset
def load_data(filename):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()[1:]  # Skip header
            data = []
            for line in lines:
                values = line.strip().split(',')
                try:
                    row = [float(v) for v in values]
                    data.append(row)
                except ValueError:
                    print("Warning: Skipping invalid row -", line.strip())
            return data
    except FileNotFoundError:
        print("Error: File not found.")
        return []

# Main execution
import math

filename = 'diabetes.csv'  # Path to your dataset
data = load_data(filename)

if data:
    # Separate features and labels
    X = [[1] + row[:-1] for row in data]  # Add bias (intercept term) as 1
    y = [int(row[-1]) for row in data]   # Class labels (0 or 1)

    weights = [0] * len(X[0])  # Initialize weights to zero

    # Train model
    weights = gradient_descent(X, y, weights, learning_rate=0.01, iterations=1000)

    # Predict
    y_pred = predict(X, weights)

    # Evaluate
    evaluate(y, y_pred)


Iteration 0 : Cost = 10.013553484350554
Iteration 100 : Cost = 4.6643031895251
Iteration 200 : Cost = 11.387249177472746
Iteration 300 : Cost = 11.788484764054123
Iteration 400 : Cost = 4.0988706920409115
Iteration 500 : Cost = 7.440055101223103
Iteration 600 : Cost = 11.652079758855402
Iteration 700 : Cost = 11.453753870853411
Iteration 800 : Cost = 9.27142466023239
Iteration 900 : Cost = 21.8135569631101
Accuracy: 0.6471
Precision: 0.4571
Recall: 0.0597
F1 Score: 0.1056
